### 참고문헌

\[1\] [17-02 버트(Bidirectional Encoder Representations from Transformers, BERT)](https://wikidocs.net/115055)  
\[2\] [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)  
\[3\] [\[BERT\] 버트 소스코드 이해](https://hyen4110.tistory.com/87)  
\[4\] [Transformers(신경망 언어모델 라이브러리) 강좌](https://wikidocs.net/book/8056)
\[5\] [A Complete Guide to BERT with Code](https://towardsdatascience.com/a-complete-guide-to-bert-with-code-9f87602e4a11/)

In [1]:
#!pip install transformers

BERT는 기본적으로 훈련된 트랜스포머 인코더 스택임.

기본적인 Bert 모델의 사용은 아래 코드와 같다.

In [2]:
from transformers import BertTokenizer, BertModel
import torch

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 1. BertTokenizer의 이해
- Tokenizer 정의 : 주어진 코퍼스(corpus)에서 토큰(token)이라 불리는 단위로 나누는 작업
- **BertTokenizer**는 무엇이 특별한가?  
  - `WordPiece Tokenizer`(BPE의 변형 알고리즘) 적용  
  - `BPE(Byte Pair Encoding)` : OOV(Out-Of-Vocabulary)문제를 완화하기위한 대표적인 서브워드 분리 일고리즘  
  - 서브워드 분리(Subword segmentation): 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들(Ex)  
     `birthplace = birth + place`의 조합으로 구성된 경우가 많기 때문에, 하나의 단어를 여러 서브워드로 분리  
     해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업 
  

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

``` python
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
```

- 토크나이징(Tokenizing)
입력된 문장을 WordPiece 방식으로 잘게 나붑니다.예)
```
tokenizer.tokenize("apple people water")
→ ['apple', 'people', 'water']
```

- 토큰을 숫자 ID로 변환(Convert tokens to IDs)
``` 
  tokenizer.convert_tokens_to_ids(['apple', 'people', 'water'])
→ [6207, 2111, 2300]   # 숫자는 vocab에 따라 다름
```

- [CLS]와 [SEP] 토큰 자동 추가
BERT는 문장의 시작과 끝에 특별한 토큰을 붙입니다.:
```
[CLS] apple people water [SEP]
```

이걸 숫자 ID로 변환한 전체 결과는 다음과 같을 수 있음.
```
[101, 6207, 2111, 2300, 102]
```
    - `101`: [CLS]
    - `102`: [SEP]

- PyTorch 텐서로 변환
return_tensors='pt' 옵션 덕분에 결과는 PyTorch 텐서로 변환됨. 예:
``` python
inputs = {
    'input_ids': tensor([[101,6207, 2111, 2300, 102]]),
    'attention_mask': tensor([[1,1,1,1]])
}
```

In [4]:
inputs 

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

### BertTokenizer의 output 해부
input_ids  
:  (torch.LongTensor of shape ({0})  
: Indices of input sequence tokens in the vocabulary.  
: Indices can be obtained using [BertTokenizer].  
: See [PreTrainedTokenizer.encode] and [PreTrainedTokenizer.__call__] for details.  

In [5]:
sequence= ["apple people water"]
inputs = tokenizer(sequence, return_tensors="pt")
  
outputs = model(**inputs)
input_ids = inputs['input_ids'] 
# tensor([[ 101, 6207, 2111, 2300,  102]])

tokenizer.decode(input_ids[0])
# [CLS] apple people water [SEP]

'[CLS] apple people water [SEP]'

![](../images/bert_token_sequence.png)

✔ token_type_ids (= segment_ids)  
: pre-training 단계에서 ‘NSP(Next Sentence Prediction)’ task를 위해 존재  
: fine-tuning 시, 모두 0 (https://ratsgo.github.io/nlpbook/docs/language_model/tutorial/)  
   
(torch.LongTensor of shape ({0}), optional)  
: Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]:  

- corresponds to a sentence A token,
- corresponds to a sentence B token.

In [6]:
sequence_a = "HuggingFace is based in NYC"
sequence_b = "Where is HuggingFace based?"

encoded_dict = tokenizer(sequence_a, sequence_b)
encoded_dict["token_type_ids"]
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

decoded = tokenizer.decode(encoded_dict["input_ids"])
# [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]


In [7]:
encoded_dict

{'input_ids': [101, 17662, 12172, 2003, 2241, 1999, 16392, 102, 2073, 2003, 17662, 12172, 2241, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
decoded

'[CLS] huggingface is based in nyc [SEP] where is huggingface based? [SEP]'